In [1]:
import arcpy
import os
from arcpy import env
from arcpy.sa import *

In [3]:
arcpy.env.workspace = r"D:\nmehmel\foot_to_set\Bld_foot\bld_pts_merged.gdb"
#files = arcpy.ListFeatureClasses("*")
files= ['AGO_bld_pts', 'BDI_bld_pts', 'BEN_bld_pts', 'BFA_bld_pts', 'BWA_bld_pts', 'COG_bld_pts', 'ESH_bld_pts', 'GHA_bld_pts', 'GIN_bld_pts', 'GMB_bld_pts', 'LSO_bld_pts', 'MOZ_bld_pts', 'MWI_bld_pts', 'NAM_bld_pts', 'NER_bld_pts', 'RWA_bld_pts', 'SOM_bld_pts', 'SWZ_bld_pts', 'TZA_bld_pts', 'UGA_bld_pts', 'CAF_bld_pts', 'CHD_bld_pts', 'CMR_bld_pts', 'DJI_bld_pts', 'GNB_bld_pts', 'MDG_bld_pts', 'CPV_bld_pts', 'ERI_bld_pts', 'ZAF_bld_pts']

In [14]:
print(files)
files = ['DJI_bld_pts', 'GNB_bld_pts', 'MDG_bld_pts', 'CPV_bld_pts', 'ERI_bld_pts', 'ZAF_bld_pts']

['GNB_bld_pts', 'MDG_bld_pts', 'CPV_bld_pts', 'ERI_bld_pts', 'ZAF_bld_pts']


In [5]:
def point_to_raster(in_points, out_raster, snap_raster):
    """create a raster that can be used to make contours and a second of building counts"""
    print(f'Converting {in_points} to a raster')

# add field for default values 
    if len(arcpy.ListFields(in_points,"tmp_cnt"))<1:
         print(f'Adding tmp field')  
         arcpy.AddField_management(in_points, field_name="tmp_cnt", field_type="LONG")
         arcpy.CalculateField_management(in_points, "tmp_cnt", "1")
    else:
        print(f'Tmp count field exists')  
# convert points to raster
    print('Creating raster')
    tmp_raster = out_raster + "_bldCounts"
    #arcpy.env.extent = snap_raster
    #arcpy.env.snapRaster = snap_raster
    arcpy.PointToRaster_conversion(in_points, "tmp_cnt", tmp_raster, "SUM", "", cellsize = 0.00095)

#convert centerpoints to points for use later (raster_points)
    raster_points= out_raster+ "raster_points"
    arcpy.conversion.RasterToPoint(tmp_raster, raster_points)
    
# reclassify nodata to zero
    print('Reclassify NoData to zero')
    max_cnt_out = arcpy.GetRasterProperties_management(tmp_raster, "MAXIMUM")
    max_cnt = float(max_cnt_out.getOutput(0))
    remap = RemapValue([["NODATA", 0]])
    outReclassify = Reclassify(tmp_raster, "value", remap, "DATA")
# save a copy for building counts
    
    outReclassify.save(tmp_raster)    
# max out the count at 150 - this makes better countours 
    print("Max out at 150")
    if max_cnt > 150:
        remap2 = RemapRange([[151, max_cnt, 150]])
        outReclassify = Reclassify(outReclassify, "value", remap2, "DATA")
        
    tmp_raster_150 = out_raster + "_bldCounts_max150"
    outReclassify.save(tmp_raster_150)
    
            

In [41]:
arcpy.env.workspace = r"D:\nmehmel\foot_to_set\Bld_foot\set_raster_100m.gdb"

for file in files:
    print(file)
    c_code = file[:3]
    print(c_code)
    in_gdb = r"D:\nmehmel\foot_to_set\Bld_foot\bld_pts_merged.gdb"
    in_pnt_file = file
    country_points = os.path.join(in_gdb, in_pnt_file)
    
    
    out_gdb = r"D:\nmehmel\foot_to_set\Bld_foot\set_raster_100m.gdb"
    out_file = c_code + "_bld_100m"
    out_filepath=os.path.join(out_gdb, out_file)
    
    extent_dir = r"D:\nmehmel\foot_to_set\raster_extents"
    extent_file = c_code.lower() + "_set_area_100m.tif"
    raster_extent = os.path.join(extent_dir, extent_file)
    
    point_to_raster(country_points, out_filepath, raster_extent)

AGO_bld_pts
AGO
Converting D:\nmehmel\foot_to_set\Bld_foot\bld_pts_merged.gdb\AGO_bld_pts to a raster
Adding tmp field
Creating raster
Reclassify NoData to zero
Max out at 150
BDI_bld_pts
BDI
Converting D:\nmehmel\foot_to_set\Bld_foot\bld_pts_merged.gdb\BDI_bld_pts to a raster
Adding tmp field
Creating raster
Reclassify NoData to zero
Max out at 150
BEN_bld_pts
BEN
Converting D:\nmehmel\foot_to_set\Bld_foot\bld_pts_merged.gdb\BEN_bld_pts to a raster
Adding tmp field
Creating raster
Reclassify NoData to zero
Max out at 150
BFA_bld_pts
BFA
Converting D:\nmehmel\foot_to_set\Bld_foot\bld_pts_merged.gdb\BFA_bld_pts to a raster
Adding tmp field
Creating raster
Reclassify NoData to zero
Max out at 150
BWA_bld_pts
BWA
Converting D:\nmehmel\foot_to_set\Bld_foot\bld_pts_merged.gdb\BWA_bld_pts to a raster
Adding tmp field
Creating raster
Reclassify NoData to zero
Max out at 150
COG_bld_pts
COG
Converting D:\nmehmel\foot_to_set\Bld_foot\bld_pts_merged.gdb\COG_bld_pts to a raster
Adding tmp field
C

In [6]:
def make_shellup_contours(in_fc, out_fc, base_contour, con_interval):
    """Create shell_up contours"""
    print(f'Making Contours for {in_fc} with a base of {base_contour} and interval of {con_interval}.')
    Contour(in_fc, out_fc, con_interval, base_contour, "", "CONTOUR_SHELL_UP")


def contour_to_singlepart(multipart_contours, out_fc, min_contour):
    """convert a multipart contour to a singlepart contour to create settlement extents"""
    print(f'making selection for a minimum contour of {min_contour}.')
    # select the minimum contour
    input_field = "ContourMin"
    input_value = min_contour
    qry = "{}= {}".format(input_field, input_value)
    select_one = arcpy.SelectLayerByAttribute_management(multipart_contours, "NEW_SELECTION", qry)
    # convert selected multipart contour to singlepart
    print('Running multipart to singlepart on the selection.')
    arcpy.MultipartToSinglepart_management(select_one, out_fc)
    
def join_bld_fp(settlements, out_fc, bld_points):
    """Join building counts to settlement extents"""
    if len(arcpy.ListFields(settlements, "bld_count")) > 0:
        print(f'Building Count field exists')
    else:
        print(f'Adding Building count field')
        arcpy.AddField_management(settlements, field_name="bld_count", field_type="LONG")
    arcpy.SpatialJoin_analysis(settlements, bld_points, out_fc, join_type="KEEP_COMMON")
    arcpy.CalculateField_management(out_fc, "bld_count", '!Join_Count!')


def drop_small_contours(in_contours, max):
    """Drop contours that are less than MAX"""
    # this does not work
    qry = "{} < {}".format("bld_count", max)
    tmp_select = arcpy.SelectLayerByAttribute_management(in_contours, "NEW_SELECTION", qry)
    print(arcpy.GetCount_management(tmp_select))
    arcpy.DeleteFeatures_management(tmp_select)


def make_simple(settlements_in, settlements_out):
    # Eliminate small holes and smooth polygons
    tmp_set = "tmp_m"
    arcpy.management.EliminatePolygonPart(settlements_in, tmp_set, "AREA", 7500, "", "CONTAINED_ONLY")
    # arcpy.env.cartographicPartitions = settlement_parts
    arcpy.cartography.SmoothPolygon(tmp_set, settlements_out, "PAEK", "100 Meters", "FIXED_ENDPOINT", "RESOLVE_ERRORS")
    #arcpy.edit.Generalize(in_features, {tolerance})



In [43]:
arcpy.env.workspace = r"D:\nmehmel\foot_to_set\Bld_foot\extents.gdb"

for file in files:
    print(file)
    c_code = file[:3]
    print(c_code)
    in_gdb = r"D:\nmehmel\foot_to_set\Bld_foot\set_raster_100m.gdb"
    in_file = c_code + "_bld_100m_bldCounts_max150"
    country_raster = os.path.join(in_gdb, in_file)
    
    out_gdb = r"D:\nmehmel\foot_to_set\Bld_foot\extents.gdb"
    out_contours = c_code + "_MP_contours"
    make_shellup_contours(country_raster, out_contours, 1, 2)
    
    out_singlepart = c_code + "_SP_contours"
    contour_to_singlepart(out_contours, out_singlepart, 1)

AGO_bld_pts
AGO
Making Contours for D:\nmehmel\foot_to_set\Bld_foot\set_raster_100m.gdb\AGO_bld_100m_bldCounts_max150 with a base of 1 and interval of 2.
making selection for a minimum contour of 1.
Running multipart to singlepart on the selection.
BDI_bld_pts
BDI
Making Contours for D:\nmehmel\foot_to_set\Bld_foot\set_raster_100m.gdb\BDI_bld_100m_bldCounts_max150 with a base of 1 and interval of 2.
making selection for a minimum contour of 1.
Running multipart to singlepart on the selection.
BEN_bld_pts
BEN
Making Contours for D:\nmehmel\foot_to_set\Bld_foot\set_raster_100m.gdb\BEN_bld_100m_bldCounts_max150 with a base of 1 and interval of 2.
making selection for a minimum contour of 1.
Running multipart to singlepart on the selection.
BFA_bld_pts
BFA
Making Contours for D:\nmehmel\foot_to_set\Bld_foot\set_raster_100m.gdb\BFA_bld_100m_bldCounts_max150 with a base of 1 and interval of 2.
making selection for a minimum contour of 1.
Running multipart to singlepart on the selection.
BWA_

In [7]:

def make_buffer(in_fc, out_fc, buffer):
    arcpy.Buffer_analysis(in_fc, out_fc, buffer)


def buffer_hamlets(in_settlements, bld_points, out_fc, buffer):
    """Create buffers around settlements that are not within a contour"""
    tmp_select = arcpy.SelectLayerByLocation_management(bld_points, "INTERSECT", in_settlements)
    hamlets = arcpy.SelectLayerByLocation_management(tmp_select, None, None, "", "SWITCH_SELECTION")
    arcpy.Buffer_analysis(hamlets, out_fc, buffer)

#new part
def buffer_raster_points(in_settlements, hamlet_buffer, raster_points, out_fc, buffer ):
    tmp_select = arcpy.SelectLayerByLocation_management(raster_points, "INTERSECT", in_settlements)
    tmp_select2 = arcpy.SelectLayerByLocation_management(tmp_select, "INTERSECT", hamlet_buffer,"", "ADD_TO_SELECTION")
    raster_pnts = arcpy.SelectLayerByLocation_management(tmp_select2, None, None, "", "SWITCH_SELECTION")
    arcpy.Buffer_analysis(raster_points, out_fc, buffer)
    arcpy.Append_management(out_fc, hamlet_buffer, "NO_TEST")
    
def merge_and_dissolve(in_settlements, hamlet_buffers, out_file):
    """merge and dissolve hamlet buffers and settlements"""
    print("Merge starting")
    arcpy.Merge_management([in_settlements, hamlet_buffers], out_file)

#    print(f"Repairing geometry before Dissolve of {out_file}")
#    arcpy.RepairGeometry_management(out_file)

    out_dissolve = out_file + "_dis"
    print("Dissolve starting")
    arcpy.Dissolve_management(out_file, out_dissolve, "", "", "SINGLE_PART")


In [15]:
arcpy.env.workspace = r"D:\nmehmel\foot_to_set\Bld_foot\extents.gdb"

for file in files:
    print(file)
    c_code = file[:3]
    print(c_code)
    in_gdb = r"D:\nmehmel\foot_to_set\Bld_foot\bld_pts_merged.gdb"
    in_pnt_file = file
    country_points = os.path.join(in_gdb, in_pnt_file)
    
    out_singlepart = c_code + "_SP_contours"
    out_buffers = c_code + "_buffer_65"
    buf_size = "65 meters"
    out_MandD = c_code + "_merge"
    
    buffer_hamlets(out_singlepart, country_points, out_buffers, buf_size)
    
    ras_gdb= r"D:\nmehmel\foot_to_set\Bld_foot\set_raster_100m.gdb"
    raster_pts = c_code+"_bld_100mraster_points"
    raster_points=os.path.join(ras_gdb, raster_pts)
    buf_raster = c_code + "_raster_buf_50"
    buffer_raster_points(out_singlepart, out_buffers, raster_points, buf_raster, "50 Meters")
    
    merge_and_dissolve(out_singlepart, out_buffers, out_MandD)
    
    
    #set_ext_final= os.path.join(r"D:\nmehmel\foot_to_set\Bld_foot\settlement_extent_final.gdb", c_code +"_set_extent")
    #make_simple(c_code+"_merge_dis", set_ext_final)
    

DJI_bld_pts
DJI
Merge starting
Dissolve starting
GNB_bld_pts
GNB
Merge starting
Dissolve starting
MDG_bld_pts
MDG
Merge starting
Dissolve starting
CPV_bld_pts
CPV
Merge starting
Dissolve starting
ERI_bld_pts
ERI
Merge starting
Dissolve starting
ZAF_bld_pts
ZAF
Merge starting
Dissolve starting


### Join Bld Cnts 

In [16]:
def join_bld_counts(settlements, out_fc, bld_points):
    # Set local variables
 
# Want to join USA cities to states and calculate the mean city population
# for each state
    targetFeatures = settlements
    joinFeatures = bld_points
 
# Output will be the target features, states, with a mean city population field (mcp)
    outfc = out_fc
 
# Create a new fieldmappings and add the two input feature classes.
    fieldmappings = arcpy.FieldMappings()
    fieldmappings.addTable(targetFeatures)
    fieldmappings.addTable(joinFeatures)
 
# First get the POP1990 fieldmap. POP1990 is a field in the cities feature class.
# The output will have the states with the attributes of the cities. Setting the
# field's merge rule to mean will aggregate the values for all of the cities for
# each state into an average value. The field is also renamed to be more appropriate
# for the output.
    pop_FieldIndex = fieldmappings.findFieldMapIndex("tmp_cnt")
    fieldmap = fieldmappings.getFieldMap(pop_FieldIndex)
 
# Get the output field's properties as a field object
    field = fieldmap.outputField
 
# Rename the field and pass the updated field object back into the field map
    field.name = "bld_count"
    field.aliasName = "bld_count"
    fieldmap.outputField = field
 
# Set the merge rule to mean and then replace the old fieldmap in the mappings object
# with the updated one
    fieldmap.mergeRule = "sum"
    fieldmappings.replaceFieldMap(pop_FieldIndex, fieldmap) 
#Run the Spatial Join tool, using the defaults for the join operation and join type
    arcpy.SpatialJoin_analysis(targetFeatures, joinFeatures, outfc, "#", "#", fieldmappings)

In [17]:
arcpy.env.workspace = r"D:\nmehmel\foot_to_set\Bld_foot\settlement_extent_final.gdb"
in_gdb = r"D:\nmehmel\foot_to_set\Bld_foot\bld_pts_merged.gdb"
in_extents_gdb = r"D:\nmehmel\foot_to_set\Bld_foot\extents.gdb"

for file in files:
    print(file)
    c_code = file[:3]
    print(c_code)
    in_pnt_file = file
    country_points = os.path.join(in_gdb, in_pnt_file)
    extent_name = c_code + "_merge_dis"
    sett_poly = os.path.join(in_extents_gdb, extent_name)
    bld_poly = c_code + "_extents_with_bld"
    join_bld_counts(sett_poly, bld_poly, country_points)

DJI_bld_pts
DJI
GNB_bld_pts
GNB
MDG_bld_pts
MDG
CPV_bld_pts
CPV
ERI_bld_pts
ERI
ZAF_bld_pts
ZAF


In [ ]:
arcpy.env.workspace = r"D:\nmehmel\foot_to_set\Bld_foot\settlement_extent_final.gdb"

for file in files:
    print(file)
    c_code = file[:3]
    print(c_code)
    out_dir = r"D:\nmehmel\foot_to_set\output"
    out_gdb = c_code + "_extents_Version_02_ALPHA.gdb"
    out_file = c_code + "_settlements_2020_V2_ALPHA"
    
    bld_poly = c_code + "_extents_with_bld"
    
    out_gdb_path = os.path.join(out_dir, out_gdb)
    out_file_path = os.path.join(out_gdb_path, out_file)
    in_poly = c_code + "_extents_with_bld"
    
    arcpy.CreateFileGDB_management(out_dir, out_gdb)

    make_simple(in_poly, out_file_path)
    arcpy.DeleteField_management(out_file_path, 
                             ["Join_Count", "TARGET_FID", "ORIG_FID", "InPoly_FID", "SmoPgnFlag", "latitude", "longitude"])

In [ ]:
### FIRST CELL ####
import mgrs

def add_id_fields(in_settlement):
    arcpy.AddField_management(in_settlement, field_name='x', field_type="DOUBLE")
    arcpy.AddField_management(in_settlement, field_name='y', field_type="DOUBLE")
    arcpy.AddField_management(in_settlement, field_name='mgrs', field_type="TEXT", field_length=50)

    arcpy.CalculateGeometryAttributes_management(in_settlement, [['x', 'INSIDE_X'], ['y', 'INSIDE_Y']],
                                                 coordinate_system="4326")
    m = mgrs.MGRS()
    with arcpy.da.UpdateCursor(in_settlement, ['y', 'x', 'mgrs']) as cursor:
        for row in cursor:
            lat = row[0]
            lon = row[1]
            mgrs_2 = m.toMGRS(lat, lon, MGRSPrecision=2)
            row[2] = mgrs_2
            try:
                cursor.updateRow(row)
            except:
                print(f'Unable to update row')

def increment_duplicates(in_settlement, in_dupes):
    i = 0
    sql = "ORDER BY " + in_dupes + " ASC"
    arcpy.AddField_management(in_settlement, field_name='d_count', field_type="TEXT", field_length=6)
    with arcpy.da.UpdateCursor(in_settlement, [in_dupes, "d_count"], sql_clause=(None, sql)) as cursor:
        for row in cursor:
            if i == 0:  # first time around
                value = row[0]
            if row[0] != value:  # if a new ID
                value = row[0]
                i = 1
            else:  # seen before just add one
                i += 1
            row[1] = i
            cursor.updateRow(row)


def add_mgrs_code(in_settlement):
    arcpy.AddField_management(in_settlement, field_name="mgrs_code", field_type="TEXT", field_length=50)
    arcpy.CalculateField_management(in_settlement, "mgrs_code", '!mgrs! + "_" + !d_count!.zfill(2)')


def find_ssa(in_settlement, min_count):
    """make any builidng over the minimum count an SSA"""
    # make a type filed if we don't have one
    if len(arcpy.ListFields(in_settlement, "type")) > 0:
        print(f'type field exists')
    else:
        print(f'Adding type field')
        arcpy.AddField_management(in_settlement, field_name="type", field_type="TEXT", field_length=50)
    # choose the minimum count that will define an SSA
    input_field = "bld_count"
    input_value = min_count
    qry = "{} >= {}".format(input_field, input_value)
    select_ssa = arcpy.SelectLayerByAttribute_management(in_settlement, "NEW_SELECTION", qry)
    arcpy.CalculateField_management(select_ssa, "type",
                                    "'Small Settlement Area'",
                                    "PYTHON3")
    
    #must convert contours to equal area projection before finding buas 
def contour_eaproj(in_contours, out_contours):
    arcpy.Project_management(in_contours, out_contours, 102022)

def find_bua(in_contours, in_settlement, area, min_count):
    """Use a combination of contours and area to find BUA's"""
    # make a type field if we don't have one
    if len(arcpy.ListFields(in_settlement, "type")) > 0:
        print(f'type field exists')
    else:
        print(f'Adding type field')
        arcpy.AddField_management(in_settlement, field_name="type", field_type="TEXT", field_length=50)
    # find conrtours larger than area
    input_field_a = "Shape_Area"
    input_value_a = area
    qry = "{}>= {}".format(input_field_a, input_value_a)
    area_select = arcpy.SelectLayerByAttribute_management(in_contours, "NEW_SELECTION", qry)
    # Use the selected contours to find BUA settlements
    area_loc_select = arcpy.SelectLayerByLocation_management(in_settlement, "INTERSECT", area_select)

    input_field_b = "bld_count"
    input_value_b = min_count
    bld_select = "{} > {}".format(input_field_b, input_value_b)
    bld_count_select = arcpy.SelectLayerByAttribute_management(area_loc_select, "ADD_TO_SELECTION", bld_select)

    arcpy.CalculateField_management(bld_count_select, "type",
                                    "'Built Up Area'",
                                    "PYTHON3")


def make_final_layers(in_settlement, settlement_type, out_gdb):
    print(f'making {settlement_type}')
    type_fc = os.path.join(out_gdb, settlement_type)
    input_field = "type"
    type_qry = "{} LIKE '%{}%' And bld_count >= 1".format(input_field, settlement_type)
    type_select = arcpy.SelectLayerByAttribute_management(in_settlement, "NEW_SELECTION", type_qry)
    arcpy.CopyFeatures_management(type_select, type_fc)
    arcpy.AddGlobalIDs_management(type_fc)
    arcpy.Generalize_edit(type_fc, "5 Meters")

In [ ]:
##### Second Cell we will need to loop this code, set the gdb ... ###
outloc = r"D:\nmehmel\foot_to_set\output"
gdblist = os.listdir(outloc)

for countrygdb in gdblist:
    ccode=countrygdb[:3]
    laststop=os.path.join(outloc, countrygdb)
    arcpy.env.workspace = laststop
    final_settl = arcpy.ListFeatureClasses()[0]
    print(final_settl)
    
     # Add global and mgrs codes
    #add_id_fields(final_settl)
    #increment_duplicates(final_settl, "mgrs")
    #add_mgrs_code(final_settl)

     #Find SSA's do this first
    #ssa_min_bld_count = 50
    #find_ssa(final_settl, ssa_min_bld_count)
    
    #project to equal area  CHANGE FINAL_SETTL BACK TO out_contours
    out_contours2= ccode + "_EAP"
    contour_eaproj(out_contours, out_contours2)
        
    # find BUA after SSA - use a minimum count of 15 with an area of 250,000
    min_contour = 13
    sp_contour_2 = ccode + "_" + str(min_contour) + "_andUp"
    contour_to_singlepart(out_contours2, sp_contour_2, min_contour)

    contour_area = 400000
    min_bld_count = 3000
    find_bua(sp_contour_2, final_settl, contour_area, min_bld_count)

    # find hamlets
    find_hamlets = arcpy.SelectLayerByAttribute_management(final_settl, "NEW_SELECTION", "type IS NULL")
    arcpy.CalculateField_management(find_hamlets, "type",
                                    "'Hamlet'",
                                    "PYTHON3")
    
    #delete temp layer:
    arcpy.Delete_management(laststop+"\\"+ccode+"_13_andUp")
    

In [31]:
#Finalize field names

#add full country name:
cntrys ={'AGO': 'Angola', 'BDI': 'Burundi', 'BEN': 'Benin', 'BFA': 'Burkina Faso', 'BWA': 'Botswana', 'CAF': 'Central African Republic', 'CIV': "Côte d'Ivoire", 'CMR': 'Cameroon', 'COD': 'Democratic Republic of the Congo', 'COG': 'Republic of the Congo', 'COM': 'Comoros', 'DJI': 'Djibuti', 'ERI': 'Eritrea', 'ESH': 'Western Sahara', 'ETH': 'Ethiopia', 'GHA': 'Ghana', 'GIN': 'Guinea', 'GMB': 'The Gambia', 'GNB':'Guinea-Bissau', 'GNQ': 'Equatorial Guinea', 'KEN': 'Kenya', 'LBR': 'Liberia', 'LSO': 'Lesotho', 'MLI': 'Mali', 'MOZ': 'Mozambique', 'MRT': 'Mauritania', 'NAM': 'Namibia', 'NER': 'Niger', 'NGA': 'Nigeria', 'REU': 'Réunion', 'RWA': 'Rwanda', 'SDN': 'Sudan', 'SEN': 'Senegal', 'SLE': 'Sierra Leone', 'SOM': 'Somalia', 'SSD': 'South Sudan', 'SWZ': 'Eswatini', 'TCD': 'Chad', 'TGO': 'Togo', 'TZA': 'Tanzania', 'UGA': 'Uganda', 'ZMB': 'Zambia', 'ZWE': 'Zimbabwe'}

for countrygdb in gdblist:
    ccode=countrygdb[:3]
    cntryname= cntrys[ccode]
    print(cntryname)
    laststop=os.path.join(outloc, countrygdb)
    arcpy.env.workspace = laststop
    final_settl = arcpy.ListFeatureClasses()[0]
    print(final_settl)
    
    arcpy.SelectLayerByAttribute_management(final_settl, "CLEAR_SELECTION")
    
    #add country name
    arcpy.AddField_management(final_settl, field_name="Country", field_type="TEXT", field_length=100)
    arcpy.CalculateField_management(final_settl, 'Country', '"' + cntryname + '"')
    
    #add country ISO:
    arcpy.AddField_management(final_settl, field_name="ISO", field_type="TEXT", field_length=10)
    arcpy.CalculateField_management(final_settl, 'ISO', '"' + ccode + '"')
    
    arcpy.management.AlterField(final_settl, 'type', 'Settlement_Type', 'Settlement_Type')
    arcpy.management.AlterField(final_settl, 'Settlement_Type', 'Type', 'Type')
    arcpy.management.AlterField(final_settl, 'mgrs_code', 'm_code', 'm_code')
    arcpy.management.AlterField(final_settl, 'm_code', 'MGRS_Code', 'MGRS_Code')
    arcpy.management.AlterField(final_settl, 'bld_count', 'Building Count', 'Building Count')
    
    #delete unnecessary fields
    arcpy.DeleteField_management(final_settl, 'd_count')
    arcpy.DeleteField_management(final_settl, 'mgrs')
    arcpy.DeleteField_management(final_settl, 'Year')
    arcpy.DeleteField_management(final_settl, 'MosaicName')
    arcpy.DeleteField_management(final_settl, 'ImgSource')
    arcpy.DeleteField_management(final_settl, 'ImgDate')
    arcpy.DeleteField_management(final_settl, 'PolySource')
    arcpy.DeleteField_management(final_settl, 'PolyDate')
    arcpy.DeleteField_management(final_settl, 'x')
    arcpy.DeleteField_management(final_settl, 'y')

Zambia
ZMB_settlements_2020_V2_ALPHA


In [32]:
newgdb= "GRID3_Zambia_Settlement_Extents_Version_02_ALPHA.gdb"
newfc= "GRID3_Zambia_Settlement_Extents_Version_02_ALPHA"
fclass = r"D:\nmehmel\foot_to_set\output\ZMB_extents_Version_02_ALPHA.gdb\ZMB_settlements_2020_V2_ALPHA"
                
#clear to be safe
arcpy.SelectLayerByAttribute_management(fclass, "CLEAR_SELECTION")
             
#export to new gdb and feature class, with correct names and field order
newfold = r"D:\nmehmel\foot_to_set\output2"
arcpy.management.CreateFileGDB(newfold, newgdb)
new_gdb_loc=os.path.join(newfold, newgdb)
                
fms = arcpy.FieldMappings()

fm = arcpy.FieldMap()
fm.addInputField(fclass,"Shape_Length")
fms.addFieldMap(fm)

fm = arcpy.FieldMap()
fm.addInputField(fclass,"Shape_Area")
fms.addFieldMap(fm)

fm = arcpy.FieldMap()
fm.addInputField(fclass,"MGRS_Code")
fms.addFieldMap(fm)
                
fm = arcpy.FieldMap()
fm.addInputField(fclass,"Country")
fms.addFieldMap(fm)
                
fm = arcpy.FieldMap()
fm.addInputField(fclass,"ISO")
fms.addFieldMap(fm)
                
fm = arcpy.FieldMap()
fm.addInputField(fclass,"Type")
fms.addFieldMap(fm)
                
fm = arcpy.FieldMap()
fm.addInputField(fclass,"Building Count")
fms.addFieldMap(fm)
               
    
            
arcpy.FeatureClassToFeatureClass_conversion(fclass, new_gdb_loc, newfc,"",fms) 

RuntimeError: FieldMap: Error in adding input field to field map